In [ ]:
import asyncio

import ipywidgets

import ipyelk

In [ ]:
sizer = ipyelk.diagram.ElkTextSizer(custom_css=[])
button = ipywidgets.Button(description="Size")
txt = ipywidgets.Text(description="Input Text")
output = ipywidgets.Label(description="Text Size")


async def call_measurement(*args):
    size = await sizer.measure(txt.value)
    output.value = f"width:{size.width} height:{size.height}"


button.on_click(lambda *args: asyncio.create_task(call_measurement()))
ipywidgets.HBox([txt, button, output])

In [ ]:
sizer.futures